# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = '85.9.KQJT873.QJT'

# the auction goes:
auction = []
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 1, 1, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 2, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])
auction = ["PAD_START","PAD_START","PAD_START"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 3, 3, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': '3D', 'insta_score': 0.861}]
[{'call': '3D', 'insta_score': 0.895}]
[{'call': '3D', 'insta_score': 0.861}]
[{'call': '3D', 'insta_score': 0.895}]


In [3]:
auction = ["PAD_START", "2D", "PASS", "PASS", "X", "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.939,
  'expected_score': 361,
  'adjustment': 470},
 {'call': '4D',
  'insta_score': 0.027,
  'expected_score': -473,
  'adjustment': -1404}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [4]:
bid.samples

['AKTx.JTxx..AK8xx xx.K8.A9xxxx.9xx QJ9xx.AQxxxx..xx 8x.9.KQJT8xx.QJT 0.06419 \n 2D-P-P-X-P-P-P (7.8)  \n 2D-P-P-X-P-P-4D-P-P-4S-P-P-P (12.5) ',
 'KQJx.AQ8x..A8xxx x.JT.A9xxxx.Kxxx AT9xxx.Kxxxxx..9 8x.9.KQJT8xx.QJT 0.05817 \n 2D-P-P-X-P-P-P (9.0)  \n 2D-P-P-X-P-P-4D-P-P-4S-P-P-P (12.5) ',
 'Q9xx.AKxx..AK9xx KT.8x.A9xxxx.8xx AJxxx.QJTxxx..xx 8x.9.KQJT8xx.QJT 0.04053 \n 2D-P-P-X-P-P-P (8.2)  \n 2D-P-P-X-P-P-4D-P-P-4S-P-P-P (12.4) ',
 'Kxx.AQxx..AK98xx ATx.8.A9xxxx.xxx QJ9xx.KJTxxxx..x 8x.9.KQJT8xx.QJT 0.03538 \n 2D-P-P-X-P-P-P (9.1)  \n 2D-P-P-X-P-P-4D-P-P-4H-P-P-P (12.1) ',
 'AQJx.KTx..AK8xxx Kx.xxx.A9xxxx.9x T9xxx.AQJ8xx..xx 8x.9.KQJT8xx.QJT 0.02696 \n 2D-P-P-X-P-P-P (8.9)  \n 2D-P-P-X-P-P-4D-P-P-4H-P-P-P (12.1) ',
 'AKQ9.AKT8..xxxxx T.Jxx.A9xxxx.K98 Jxxxxx.Qxxxx..Ax 8x.9.KQJT8xx.QJT 0.02481 \n 2D-P-P-X-P-P-P (9.7)  \n 2D-P-P-X-P-P-4D-P-P-4S-P-P-P (12.0) ',
 'JTx.AKTxx..AK9xx Qxx.xx.A9xxxx.xx AK9xx.QJ8xx..8xx 8x.9.KQJT8xx.QJT 0.02480 \n 2D-P-P-X-P-P-P (8.4)  \n 2D-P-P-X-P-P-4D-P-P-X-P-

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [5]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

8x.9.KQJT8xx.QJT JTxxx.JTxx.x.Kxx AKQ9.AKQx.Ax.A8x xx.8xxx.9xx.9xxx
8x.9.KQJT8xx.QJT 9xxx.KTxxx.x.xxx AKTx.AJ8x.A9.AKx QJx.Qxx.xxx.98xx
8x.9.KQJT8xx.QJT KJT9.T8xxx..98xx AQxx.KQ.Axx.AKxx xxx.AJxxx.9xx.xx
8x.9.KQJT8xx.QJT J9x.Q8xxx.xx.9xx AKQT.AKxx.x.AK8x xxxx.JTx.A9x.xxx
8x.9.KQJT8xx.QJT Qxxx.QTxx.x.9xxx AKJx.KJ8x.Ax.AKx T9x.Axxx.9xx.8xx
8x.9.KQJT8xx.QJT QJxx.AJxx.9x.98x AK9x.KQT.Ax.AKxx Txx.8xxxx.xx.xxx
8x.9.KQJT8xx.QJT QTxx.QJT8x.x.8xx AKJx.Kxx.A9.AKxx 9xx.Axxx.xxx.9xx
8x.9.KQJT8xx.QJT xx.QJTxx.x.8xxxx AKxx.Axx.Ax.AKxx QJT9x.K8xx.9xx.9
8x.9.KQJT8xx.QJT Txx.KTxxx.xx.xxx AKJx.A8x.A9.AK9x Q9xx.QJxx.xx.8xx
8x.9.KQJT8xx.QJT KTxx.AJTxx.xx.xx AQJ9.x.A9x.AKxxx xxx.KQ8xxx.x.98x
